# Using paperXai to get your personal arXiv daily digest

The goal of this notebook is to help you use this package and to understand the different components of the pipeline. Briefly, the pipeline can be decomposed into the following sections which will correspond to the sections of this notebook.

- [Fetching the latest arXiv papers](#fetching-the-latest-arxiv-papers)
- [Embedding predefined user questions and sections](#embedding-predefined-user-questions-and-sections)
- [Semantic retrieval](#semantic-retrieval)
- [Generating an automatic report](#generating-an-automatic-report)
- [Sending the personalized newsletter](#sending-the-personalized-newsletter)

For any comments, feel free to reach out directly (see [here](https://sebastianpartarrieu.github.io/)), or via an issue on the github repository.

In [1]:
# initial imports

import pandas as pd
import paperxai.constants as constants

## Fetching the latest arXiv papers

In [47]:
df_articles = pd.read_csv(constants.ROOT_DIR + "/data/arxiv/base_papers.csv")

In [48]:
df_articles

,Title,URL,Abstract,Authors,Published Date,Category,Paper ID
0,Data-driven criteria for quantum correlations,http://arxiv.org/abs/2307.11091v1,We build a machine learning model to detect co...,"Mateusz Krawczyk, Jarosław Pawłowski, Maciej M...",2023-07-20T17:59:59Z,quant-ph,2307.11091v1
1,L-Eval: Instituting Standardized Evaluation fo...,http://arxiv.org/abs/2307.11088v1,"Recently, there has been growing interest in e...","Chenxin An, Shansan Gong, Ming Zhong, Mukai Li...",2023-07-20T17:59:41Z,cs.CL,2307.11088v1
2,PAPR: Proximity Attention Point Rendering,http://arxiv.org/abs/2307.11086v1,Learning accurate and parsimonious point cloud...,"Yanshu Zhang, Shichong Peng, Alireza Moazeni, ...",2023-07-20T17:59:33Z,cs.CV,2307.11086v1
3,Representation Learning in Anomaly Detection: ...,http://arxiv.org/abs/2307.11085v1,"In this perspective paper, we argue that the d...",Yedid Hoshen,2023-07-20T17:59:11Z,cs.LG,2307.11085v1
4,"GLSFormer: Gated - Long, Short Sequence Transf...",http://arxiv.org/abs/2307.11081v1,Automated surgical step recognition is an impo...,"Nisarg A. Shah, Shameema Sikder, S. Swaroop Ve...",2023-07-20T17:57:04Z,cs.CV,2307.11081v1
...,...,...,...,...,...,...,...
995,Opinion mining using Double Channel CNN for Re...,http://arxiv.org/abs/2307.07798v1,Much unstructured data has been produced with ...,"Minoo Sayyadpour, Ali Nazarizadeh",2023-07-15T13:11:18Z,cs.IR,2307.07798v1
996,Joint Adversarial and Collaborative Learning f...,http://arxiv.org/abs/2307.07791v1,Considering the instance-level discriminative ...,"Tianyu Guo, Mengyuan Liu, Hong Liu, Wenhao Li,...",2023-07-15T12:37:18Z,cs.CV,2307.07791v1
997,Adaptive Nonlinear Latent Transformation for C...,http://arxiv.org/abs/2307.07790v1,Recent works for face editing usually manipula...,"Zhizhong Huang, Siteng Ma, Junping Zhang, Hong...",2023-07-15T12:36:50Z,cs.CV,2307.07790v1
998,The Interpolating Information Criterion for Ov...,http://arxiv.org/abs/2307.07785v1,The problem of model selection is considered f...,"Liam Hodgkinson, Chris van der Heide, Robert S...",2023-07-15T12:09:54Z,stat.ML,2307.07785v1


## Embedding predefined user questions and sections

## Semantic retrieval

## Generating an automatic report

## Sending the personalized newsletter